In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [27]:
data = pd.read_csv('monkeypox_cleaned.csv')

In [28]:
X = data[['Systemic Illness', 'Rectal Pain', 'Sore Throat', 'Penile Oedema', 'Oral Lesions',
          'Solitary Lesion', 'Swollen Tonsils', 'HIV Infection', 'Red blood cells', 'White blood cells',
          'Age', 'Sexually Transmitted Infection']]
y = data['MPOX']

In [29]:
le = LabelEncoder()

In [ ]:
categorical_features = ['Systemic Illness', 'Rectal Pain', 'Sore Throat', 'Penile Oedema', 'Oral Lesions',
                        'Solitary Lesion', 'Swollen Tonsils', 'HIV Infection', 'Sexually Transmitted Infection']
for feature in categorical_features:
    X[feature] = le.fit_transform(X[feature])

In [31]:
test_size = 0.2
random_state = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

In [ ]:
print("Feature names used for building the classification models:")
print(X.columns.tolist())

In [ ]:
print("\nShape of the training set (X_train):", X_train.shape)
print("Shape of the training set (y_train):", y_train.shape)
print("Shape of the test set (X_test):", X_test.shape)
print("Shape of the test set (y_test):", y_test.shape)

In [34]:
lr_model = LogisticRegression(random_state=random_state)
dt_model = DecisionTreeClassifier(random_state=random_state)
knn_model = KNeighborsClassifier()
svm_model = SVC(kernel='rbf', random_state=random_state)
nb_model = GaussianNB()

models = [lr_model, dt_model, knn_model, svm_model, nb_model]

for model in models:
    model.fit(X_train, y_train)